In [3]:
print('Hello world')

Hello world


In [2]:
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
# S, K, risk, periods, rate_i, rate_dy, 
# option_type, exercise_style, 

In [117]:
class BinPriMod:
    def __init__(self, S, K, sigma, periods, h, rate_i, rate_dy,
                 option_type, exercise_style, txt_shift):
        self.spot = S
        self.strike = K
        self.risk = sigma
        self.periods = periods
        self.time_span = h
        self.risk_free = rate_i
        self.dividend_yield = rate_dy
        self.option_type = option_type
        self.exercise_style = exercise_style
        self.period_length = self.periods / self.time_span
        self.text_shift = txt_shift
        self.values = None
        self.prices = None


    def help():
        print('def __init__(self, S, K, sigma, periods, h, rate_i, rate_dy,option_type, exercise_style):')
        print('\n')
        print('Commands:\nup_and_down,\nstorage_values,\noption_price')

    def up_and_down(self, spot, h):   
        uS = spot*np.e**((self.risk_free - self.dividend_yield) * 
                              h + self.risk*np.sqrt(h)) 
        dS = spot*np.e**((self.risk_free -self.dividend_yield) * 
                              h - self.risk*np.sqrt(h))
        return [uS, dS]

    def storage_values(self):
        storage = [[self.spot]]
        for i in range(0, self.periods,1):
            sub_storage = []
            for j in range(0, len(storage[i]),2):   
                sub_storage.append(np.array(self.up_and_down(storage[i][j],
                                                             self.time_span)))
            if (i+1)>=2:
                if (i+1)%2== 0:
                    sub_storage.append(np.reshape(self.up_and_down(storage[i][j+1],
                                                                   self.time_span)[1],(1,)))
            fixed = np.concatenate(sub_storage)
            storage.append(np.round(fixed,3))
        self.values = storage
        return self.values 


    def option_price(self, 
                     up, down, high_optpri, low_optpri):
        u = up/self.spot
        d = down/self.spot
        p = (np.e**((self.risk_free-self.dividend_yield)*self.time_span)-d)/(u-d)

        opt_price = np.e**(-self.risk_free*self.time_span) * ((high_optpri*p)+(low_optpri*(1-p)))

        if self.exercise_style == 'European':
            opt_price = opt_price
        elif self.exercise_style == 'American':
            if self.option_type =='Put':
                opt_price = max(opt_price, self.strike -self.spot)
            else:
                opt_price = max(opt_price, self.spot-self.strike)            
        else:
            print('State exercise style!!!')
        return round(opt_price,3)
    
    def storage_prices(self):

        latest_opt_pri = []
        print(self.values)

        if self.option_type=='Call':
            array = np.where(self.values[self.periods]-self.strike>0,self.values[self.period_length]-self.strike,0)
        elif self.option_type=='Put':
            array = np.where(self.values[self.period_length]-self.strike<0,self.values[self.period_length]-self.strike,0)
        else:
            print('State option type!!!')
        latest_opt_pri.append(abs(array))

        for i in range(0, self.period_length, 1): #periods -1
            sub_optpri_storage = []
            for j in range(0,self.period_length-i,1):   #periods- 1
                sub_optpri_storage.append(self.option_price(self,
                                                            self.values[self.period_length][-j-1], 
                                                            self.values[self.period_length][-j-2],
                                                            latest_opt_pri[i][-1-j], 
                                                            latest_opt_pri[i][-2-j]))
            latest_opt_pri.append(np.flip(np.round(sub_optpri_storage,2)))
        self.prices = np.flip(latest_opt_pri)
    
    

    def plot_bin_tree(self):
    

        fig, ax = plt.subplots(figsize=(6,6))
        ax.scatter(x=[0], y=[self.values[0][0]])
        for i in range(0,int(self.period_length)+1,1):
            ax.scatter(x=np.repeat((i)*self.time_span,i+1), y= self.values[i])
            for j in range(0,i+1,1):
                text_value = 'Value '+ str(self.values[i][j])
                text_price = 'Price '+ str(self.prices[i][j])
                ax.annotate(text_value, xy=(i*self.time_span, self.values[i][j]+self.text_shift))
                ax.annotate(text_price, xy=(i*self.time_span, self.values[i][j]-self.text_shift))
            ax.set_xlabel('Months')
        ax.set_ylabel('Price')
        plt.xlim(-2,self.periods+2*self.time_span)
        plt.title('Binomial tree for '+self.exercise_style+' '+self.option_type+' option')
        plt.show()



In [63]:
BinPriMod.help()

def __init__(self, S, K, sigma, periods, h, rate_i, rate_dy,option_type, exercise_style):


Commands:
up_and_down,
storage_values,
option_price


In [104]:
boy = BinPriMod(50, 52, 0.3, 3, 1, 0.03, 0.04, 'Call', 'Put',3)
boy.storage_values()

[[50],
 array([66.821, 36.672]),
 array([89.301, 49.01 , 26.897]),
 array([119.344,  65.498,  35.946,  19.728])]

In [118]:
boy.storage_prices()

[[50], array([66.821, 36.672]), array([89.301, 49.01 , 26.897]), array([119.344,  65.498,  35.946,  19.728])]


AttributeError: 'BinPriMod' object has no attribute 'storage'